In [4]:
import numpy as np
import pandas as pd
import glob
import os

In [49]:
# change it to your frame folder
frame_folder = '/home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150'

In [50]:
frame_filepathlist = glob.glob(frame_folder + "/*.frame")
len(frame_filepathlist)

122

## Generate overlapped prediction 
(Could be ingore)

In [101]:
def gen_overlap_seq(frame_filepath, method, overlap=0.875, seg_len=0.63, shift=0.01, out_dir = None):
    
    frame = np.loadtxt(frame_filepath)
    name = os.path.basename(frame_filepath).split(".frame")[0] + "." + method
    
    if not out_dir:
        out_dir = os.path.dirname(frame_filepath) + "+" + method + "_" + str(overlap)
        if not os.path.exists(out_dir):
            os.mkdir(out_dir)
    overlap_filepath = os.path.join(out_dir, name)

    print(overlap_filepath)

    seg = int((seg_len/shift+1))
    jump = int(seg * (1-overlap))

    all_out = np.empty([len(frame), ])

    i=0
    out = 0
    while(i<len(frame)):
        idx = [i] + np.arange(0, seg, jump)
        arr = []
        for j in idx:
            if j>=0 and j < len(frame):
                arr.append(frame[j])
            else:
                arr.append(np.nan)
        if method == "median":     
            out = np.nanmedian(arr)
        elif method == "mean":
            out = np.nanmean(arr)
        else:
            raise error("method should be either mean or median")
    
        all_out[i] = out
        i+=1
        
    # save result
    with open(overlap_filepath, "w") as fout:
        for m in all_out:
            fout.write('{0:0.2f}\n'.format(m))
            
    return all_out

In [102]:
method = 'mean'

for frame_filepath in frame_filepathlist:
    seq = gen_overlap_seq(frame_filepath, method, overlap = 0.125)

/home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150+mean_0.125/xmqSaQPzL1E.mean
/home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150+mean_0.125/U_WzY2k8IBM.mean
/home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150+mean_0.125/vfjywN5CN0Y.mean
/home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150+mean_0.125/B1MAUxpKaV8.mean
/home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150+mean_0.125/TzaVHtLXOzY.mean
/home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150+mean_0.125/hHgg9WI8dTk.mean
/home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150+mean_0.125/lDmLcWWBp1E.mean
/home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150+mean_0.125/8aMv-ZGD4ic.mean
/home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150+mean_0.125/9Y_l9NsnYE0.mean
/home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150+mean_0.125/_a9SWtc

/home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150+mean_0.125/tghXjom3120.mean
/home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150+mean_0.125/G5Yr20A5z_Q.mean
/home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150+mean_0.125/zC5Fh2tTS1U.mean
/home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150+mean_0.125/a5mEmM6w_ks.mean
/home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150+mean_0.125/JNb4nWexD0I.mean
/home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150+mean_0.125/Ksd1JQFHYWA.mean
/home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150+mean_0.125/WwoTG3_OjUg.mean
/home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150+mean_0.125/Ov0za6Xb1LM.mean
/home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150+mean_0.125/55Ihr6uVIDA.mean
/home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150+mean_0.125/2PpxiG0

## Write to RTTM format

In [105]:
def gen_seg_dict(sequence, threshold, seg_len=0.63, shift = 0.01):
    
    first = seg_len/2
    start = 0
    end = 0
    start_list = [first]
    end_list = []
    state_list = []
    
    for i in range(len(sequence)-1):
        current_sate = "non-speech" if sequence[i] <= threshold else "speech"
        next_state =  "non-speech" if sequence[i+1] <= threshold else "speech"
        if next_state != current_sate:
            end = first + i*shift + shift # shift for handling joint
            state_list.append(current_sate)
            end_list.append(end)

            start = first + (i+1)*shift
            start_list.append(start)

    end_list.append((i+1)*shift + first + shift) 
    state_list.append(current_sate)

    rttm_dict = {'start': start_list, 'end': end_list, 'vad': state_list} 
    return pd.DataFrame(rttm_dict) 

In [106]:
threshold = 0.9

rttm_folder ='vad_rttm' + "_" + str(threshold) 
if not os.path.exists(rttm_folder):
    os.mkdir(rttm_folder)


for frame_filepath in frame_filepathlist:
    print(f"process {frame_filepath}")
    name = frame_filepath.split("/")[-1].split(".frame")[0]
    frame_seq = np.loadtxt(frame_filepath)
    
    # call gen_overlap_seq to generate overlapped prediction results
    # mean_seq = gen_overlap_seq(frame_filepath, method, overlap = 0.875)

    # generate rttm file
    df = gen_seg_dict(frame_seq, threshold) #  change it if want use overlapped prediction

    method = 'frame'
    save_name = name + "."  +  method + ".rttm"
    save_path = os.path.join(rttm_folder, save_name)
    print(save_path)
    df.to_csv(save_path, sep='\t', index=False, header=False)

process /home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150/xmqSaQPzL1E.frame
vad_rttm_0.9/xmqSaQPzL1E.frame.rttm
process /home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150/U_WzY2k8IBM.frame
vad_rttm_0.9/U_WzY2k8IBM.frame.rttm
process /home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150/vfjywN5CN0Y.frame
vad_rttm_0.9/vfjywN5CN0Y.frame.rttm
process /home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150/B1MAUxpKaV8.frame
vad_rttm_0.9/B1MAUxpKaV8.frame.rttm
process /home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150/TzaVHtLXOzY.frame
vad_rttm_0.9/TzaVHtLXOzY.frame.rttm
process /home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150/hHgg9WI8dTk.frame
vad_rttm_0.9/hHgg9WI8dTk.frame.rttm
process /home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150/lDmLcWWBp1E.frame
vad_rttm_0.9/lDmLcWWBp1E.frame.rttm
process /home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_f

vad_rttm_0.9/1j20qq1JyX4.frame.rttm
process /home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150/U6m3kNFjdTs.frame
vad_rttm_0.9/U6m3kNFjdTs.frame.rttm
process /home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150/yo-Kg2YxlZs.frame
vad_rttm_0.9/yo-Kg2YxlZs.frame.rttm
process /home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150/PmElx9ZVByw.frame
vad_rttm_0.9/PmElx9ZVByw.frame.rttm
process /home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150/9F2voT6QWvQ.frame
vad_rttm_0.9/9F2voT6QWvQ.frame.rttm
process /home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150/skiZueh4lfY.frame
vad_rttm_0.9/skiZueh4lfY.frame.rttm
process /home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150/phVLLTMzmKk.frame
vad_rttm_0.9/phVLLTMzmKk.frame.rttm
process /home/fjia/code/NeMo-fei/examples/asr/SAD/mbn_frame/3x1_sgd/sgd_0.01_150/kMy-6RtoOVU.frame
vad_rttm_0.9/kMy-6RtoOVU.frame.rttm
process /home/fjia/

vad_rttm_0.9/fD6VkIRlIRI.frame.rttm
